In [9]:
import numpy as np
import keras 

np.random.seed(1337)

# ddqn replay:
batch = [
    # state, action, rew, state', term
    (np.array([ 0.0019983 ,  0.04791403, -0.04181988,  0.01041538]), 0, 1, np.array([0.00295658, -0.14658399, -0.04161158,  0.28961586]), False),
    (np.array([-0.03828615,  0.03984183, -0.03534159,  0.03724982]), 1, 1, np.array([-0.03748931,  0.23545229, -0.03459659, -0.26637091]), False),
    (np.array([-0.03681138, -0.00375988, -0.01147762, -0.02528765]), 0, 1, None, True)     
]


def make_network():
    q_network = keras.Sequential()
    q_network.add(keras.layers.Dense(64, activation='relu', input_shape=[4]))
    q_network.add(keras.layers.Dense(2, activation='linear'))    
    q_network.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    return q_network


# Reference implementation
GAMMA = 0.99


def getTargets(batch):
    state_size = 4
    action_size = 2
    no_state = np.zeros(state_size)
    target_net = make_network()
    online_net = make_network()
    target_net.set_weights(online_net.get_weights())
    
    states = np.array([ o[0] for o in batch ])
    states_ = np.array([ (no_state if o[3] is None else o[3]) for o in batch ])

    # p = agent.brain.predict(states)
    p = online_net.predict(states)

    # p_ = agent.brain.predict(states_, target=False)
    p_ = online_net.predict(states_)
    # pTarget_ = agent.brain.predict(states_, target=True)
    pTarget_ = target_net.predict(states_)

    x = np.zeros((len(batch), state_size))
    y = np.zeros((len(batch), action_size))
    errors = np.zeros(len(batch))
    import pdb; pdb.set_trace()
    for i in range(len(batch)):
        o = batch[i]
        s = o[0]; a = o[1]; r = o[2]; s_ = o[3]

        t = p[i]
        oldVal = t[a]
        if s_ is None:
            t[a] = r
        else:
            t[a] = r + GAMMA * pTarget_[i][ np.argmax(p_[i]) ]  # double DQN

        x[i] = s
        y[i] = t
        errors[i] = abs(oldVal - t[a])
    print(y)
    return (x, y, errors)

getTargets(batch)

[21] > <ipython-input-9-ce2284abfd89>(50)getTargets()
-> for i in range(len(batch)):
(Pdb++) n
[21] > <ipython-input-9-ce2284abfd89>(51)getTargets()
-> o = batch[i]
(Pdb++) n
[21] > <ipython-input-9-ce2284abfd89>(52)getTargets()
-> s = o[0]; a = o[1]; r = o[2]; s_ = o[3]
(Pdb++) n
[21] > <ipython-input-9-ce2284abfd89>(54)getTargets()
-> t = p[i]
(Pdb++) n
[21] > <ipython-input-9-ce2284abfd89>(55)getTargets()
-> oldVal = t[a]
(Pdb++) n
[21] > <ipython-input-9-ce2284abfd89>(56)getTargets()
-> if s_ is None:
(Pdb++) n
[21] > <ipython-input-9-ce2284abfd89>(59)getTargets()
-> t[a] = r + GAMMA * pTarget_[i][ np.argmax(p_[i]) ]  # double DQN
(Pdb++) a
0
(Pdb++) np.argmax(p_[i])
0
(Pdb++) i
0
(Pdb++) p_
array([[-0.01764342, -0.09789381],
       [ 0.05493462, -0.08499158],
       [ 0.        ,  0.        ]], dtype=float32)
(Pdb++) c
[[ 0.98253304 -0.00576293]
 [-0.01065778  1.0543853 ]
 [ 1.         -0.00305891]]


(array([[ 0.0019983 ,  0.04791403, -0.04181988,  0.01041538],
        [-0.03828615,  0.03984183, -0.03534159,  0.03724982],
        [-0.03681138, -0.00375988, -0.01147762, -0.02528765]]),
 array([[ 0.98253304, -0.00576293],
        [-0.01065778,  1.0543853 ],
        [ 1.        , -0.00305891]]),
 array([0.99440372, 1.06530678, 0.9947474 ]))